In [ ]:
import os
import copy

import numpy as np
import healpy as hp

import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker 

from astropy.time import Time, TimeDelta
import astropy.units as u
from rubin_scheduler.utils import Site
from astropy.coordinates import SkyCoord, AltAz
from astroplan import Observer
from rubin_scheduler.utils import pseudo_parallactic_angle, approx_ra_dec2_alt_az, approx_altaz2pa, rotation_converter
from rubin_scheduler.utils import angular_separation

import astropy
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'


from lsst_efd_client import EfdClient

from lsst.summit.utils import ConsDbClient
import requests


# USDF setup
efd = 'usdf_efd'  
# Not necessary for slewtime? 
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"
os.environ["LSST_CONSDB_PQ_URL"] = "http://consdb-pq.consdb:8080/consdb"
os.environ["no_proxy"] += ",.consdb"

from rubin_scheduler.scheduler.model_observatory import KinemModel, tma_movement, rotator_movement

from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState

In [ ]:
async def get_rotator_limits(t_start: Time, t_end: Time, efd_client: EfdClient) -> pd.DataFrame:
    # Get rotator limit information
    topic = 'lsst.sal.MTRotator.logevent_configuration'
    rot_mapping = {'positionAngleLowerLimit': 'rotator_min',
                   'positionAngleUpperLimit': 'rotator_max',
                   'velocityLimit': 'maxspeed',
                   'accelerationLimit': 'accel',
                   'emergencyJerkLimit': 'jerk',
                   'drivesEnabled': 'drivesEnabled'}
    fields = list(rot_mapping.keys())
    rot = await efd_client.select_time_series(topic, fields, t_start, t_end)
    rot.query('drivesEnabled == 1.0', inplace=True)
    ts = t_start
    while len(rot) == 0 or rot.index.min() > t_start:
        # Go further back in time to find the state
        ts = ts - TimeDelta(3, format='jd')
        rot = await efd_client.select_time_series(topic, fields, ts, t_end)
        rot.query('drivesEnabled == 1.0', inplace=True)
    rot.rename(rot_mapping, axis=1, inplace=True)
    # Make sure a value is in place for t_start
    index_edges = [pd.to_datetime(t_start.utc.datetime).tz_localize("UTC"),
                  pd.to_datetime(t_end.utc.datetime).tz_localize("UTC")]
    rot_edges = pd.DataFrame(np.nan, index=index_edges, columns=list(rot_mapping.values()))
    rot = pd.concat([rot, rot_edges])
    rot.sort_index(inplace=True)
    rot.drop('drivesEnabled', axis=1, inplace=True)
    rot.ffill(axis=0, inplace=True)
    rot.query('index >= @index_edges[0] and index <= @index_edges[1]', inplace=True)
    return rot

async def get_tma_limits(t_start: Time, t_end: Time, efd_client: EfdClient) -> pd.DataFrame:
    # Get elevation limits
    topic = 'lsst.sal.MTMount.logevent_elevationControllerSettings'
    el_mapping = {'minL1Limit': 'altitude_minpos',
              'maxL1Limit': 'altitude_maxpos',
              'maxMoveVelocity': 'altitude_maxspeed',
              'maxMoveAcceleration': 'altitude_accel',
              'maxMoveJerk': 'altitude_jerk'}
    fields = list(el_mapping.keys())
    elevation = await efd_client.select_time_series(topic, fields, t_start, t_end)
    ts = t_start
    while len(elevation) == 0 or elevation.index.min() > t_start:
        # Go further back in time to find the state
        ts = ts - TimeDelta(1, format='jd')
        elevation = await efd_client.select_time_series(topic, fields, ts, t_end)
    elevation.rename(el_mapping, axis=1, inplace=True)
    # Get azimuth limits
    topic = 'lsst.sal.MTMount.logevent_azimuthControllerSettings'
    az_mapping = {'minL1Limit': 'azimuth_minpos',
              'maxL1Limit': 'azimuth_maxpos',
              'maxMoveVelocity': 'azimuth_maxspeed',
              'maxMoveAcceleration': 'azimuth_accel',
              'maxMoveJerk': 'azimuth_jerk'}
    fields = list(az_mapping.keys())
    azimuth = await efd_client.select_time_series(topic, fields, t_start, t_end)
    ts = t_start
    while len(azimuth) == 0 or azimuth.index.min() > t_start:
        # Go further back in time to find the state
        ts = ts - TimeDelta(1, format='jd')
        azimuth = await efd_client.select_time_series(topic, fields, ts, t_end)
    azimuth.rename(az_mapping, axis=1, inplace=True)
    # First be sure we can come up with a value in place for t_start
    index_edges = [pd.to_datetime(t_start.utc.datetime).tz_localize("UTC"),
                  pd.to_datetime(t_end.utc.datetime).tz_localize("UTC")]
    elevation_edges = pd.DataFrame(np.nan, index=index_edges, columns=list(el_mapping.values()))
    azimuth_edges = pd.DataFrame(np.nan, index=index_edges, columns=list(az_mapping.values()))
    elevation = pd.concat([elevation, elevation_edges])
    elevation.sort_index(inplace=True)
    # Fill nans with previous values
    elevation.ffill(axis=0, inplace=True)
    azimuth = pd.concat([azimuth, azimuth_edges])
    azimuth.sort_index(inplace=True)
    # Fill nans with previous values
    azimuth.ffill(axis=0, inplace=True)
    # Merge these together with a 10 second tolerance
    match_range = pd.Timedelta(10, unit='second')
    tma = pd.merge_asof(left=elevation, right=azimuth, left_index=True, right_index=True, direction='nearest', tolerance=match_range)
    # And another fill, where azimuth was updated without altitude, etc.
    tma.ffill(axis=0, inplace=True)
    tma.query('index >= @index_edges[0] and index <= @index_edges[1]', inplace=True)
    return tma
    
efd_client = EfdClient('usdf_efd')
t_start = Time('2024-10-24T00:00:00')
t_end = Time('2024-12-12T00:00:00')
#t_end = t_start + TimeDelta(1, format='jd')
print(t_start.utc.iso, t_end.utc.iso)
tma = await get_tma_limits(t_start, t_end, efd_client)

In [ ]:
t_start = Time(visits.query('day_obs == 20241025')['obs_start_mjd'].min(), format='mjd', scale='tai').utc
t_end = Time(visits.query('day_obs == 20241025')['obs_start_mjd'].max(), format='mjd', scale='tai').utc
tma = await get_tma_limits(t_start, t_end, efd_client)
dict(tma.median())

In [ ]:
tma_movement(1)

In [ ]:
maxval = tma_movement(1)
plt.figure(figsize=(10, 6))
plt.plot(tma.index, tma.altitude_maxspeed/maxval['azimuth_maxspeed'])
plt.plot(tma.index, tma.azimuth_maxspeed/maxval['azimuth_maxspeed'])
plt.tick_params("x", rotation=45)
plt.ylabel("TMA max velocity (percent)")
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(2)) 
plt.grid(alpha=0.3)
list(zip(tma.index, tma['azimuth_maxspeed']/maxval['azimuth_maxspeed']))

In [ ]:
def get_visits(day_obs_int_min, day_obs_int_max):
    
    consdb = ConsDbClient()
    instrument = 'lsstcomcam'
    
    # Querying separately and joining in pandas works 
    visit_query = f'''
        SELECT * 
        FROM cdb_{instrument}.visit1
         WHERE day_obs >= {day_obs_int_min}
         and day_obs  <= {day_obs_int_max} 
    '''
    
    quicklook_query = f'''
        SELECT q.*  FROM cdb_{instrument}.visit1_quicklook as q,
        cdb_{instrument}.visit1 as v
         WHERE q.visit_id = v.visit_id and 
         v.day_obs >= {day_obs_int_min} 
         and v.day_obs <= {day_obs_int_max}
    '''
    
    try:
        visits = consdb.query(visit_query).to_pandas()
    except requests.HTTPError or requests.JSONDecodeError:
        # Try twice
        visits = consdb.query(visit_query).to_pandas()
        
    visits.set_index('visit_id', inplace=True)
    
    if len(visits) > 0:
        display(Markdown(f"Retrieved {len(visits)} visits from consdb"))
    
    quicklook = consdb.query(quicklook_query).to_pandas()
    
    if len(quicklook) > 0:
        quicklook.set_index('visit_id', inplace=True)
        visits = visits.join(quicklook, lsuffix='', rsuffix='_q')
        display(Markdown(f"And added quicklook stats"))
    
    if len(visits) == 0:
        display(Markdown(f"No visits for {telescope} between {day_obs_min} to {day_obs_max} retrieved from consdb"))
    
    values = (dict([[e,""] for e in ['science_program','target_name', 'observation_reason']]))
    visits.fillna(value=values, inplace=True)
    
    columns_to_floats = ['s_ra', 's_dec', 'exp_midpt_mjd', 'airmass', 'zero_point_median', 'psf_sigma_median', 'sky_bg_median']
    for col in columns_to_floats:
        visits[col] = visits[col].astype('float')
    
    visits.sort_values(by='exp_midpt_mjd', inplace=True)

    # For slewtime calculations 
    prev_visit_end = np.concatenate([np.array([0]), visits.obs_end_mjd[0:-1]])
    delta_t = np.concatenate([np.array([0]), (visits.obs_start_mjd[1:].values - visits.obs_end_mjd[:-1].values) * 24 * 60 * 60])
    visits['prev_obs_end_mjd'] = prev_visit_end
    visits['delta_t'] = delta_t

    # Add in physical rotator angle, position angle, and mean rotator tracking speed during the visit 
    lsst_loc = Site('LSST')
    rc = rotation_converter(telescope="comcam")
    alt, az = approx_ra_dec2_alt_az(visits.s_ra.values, visits.s_dec.values, lsst_loc.latitude, lsst_loc.longitude, visits.exp_midpt_mjd.values, lmst=None)
    pa = approx_altaz2pa(alt, az, lsst_loc.latitude)
    rotTelPos = rc.rotskypos2rottelpos(visits.sky_rotation.values, pa)
    visits['approx_pa'] = pa
    visits['approx_rotTelPos'] = rotTelPos
    
    # lsst_observer = Observer(location=lsst_loc.to_earth_location(), timezone="Chile/Continental", name="Rubin")
    
    # def add_rotator_angle_tracking_speed(x, lsst_observer, rc):
    #     skycoords = SkyCoord(x.s_ra * u.degree, x.s_dec * u.degree, frame='icrs')
    #     tstart = Time(x.obs_start_mjd, scale='tai', format='mjd')
    #     altaz_start = lsst_observer.altaz(tstart, skycoords)
    #     pa_start = lsst_observer.parallactic_angle(tstart, skycoords)
    #     x.pa_start = pa_start.degree
    #     tend = Time(x.obs_end_mjd, scale='tai', format='mjd')
    #     altaz_end = lsst_observer.altaz(tend, skycoords)
    #     pa_end = lsst_observer.parallactic_angle(tend, skycoords)
    #     x.pa_end = pa_end.degree
    #     x.rotTelPos_start = rc.rotskypos2rottelpos(x.sky_rotation, x.pa_start)
    #     x.rotTelPos_end = rc.rotskypos2rottelpos(x.sky_rotation, x.pa_end)
    #     return x
        
    # visits['pa_start'] = np.zeros(len(visits), float)
    # visits['pa_end'] = np.zeros(len(visits), float)
    # visits['rotTelPos_start'] = np.zeros(len(visits),float)
    # visits['rotTelPos_end'] = np.zeros(len(visits), float)
    # visits = visits.apply(add_rotator_angle_tracking_speed, args=[lsst_observer, rc], axis=1)
    # visits['rotTelPos_vel'] = (visits['rotTelPos_end'] - visits['rotTelPos_start']) / ((visits.obs_end_mjd - visits.obs_start_mjd) * 24 * 60 * 60)
    
    return visits


async def get_slewtimes(day_obs_list, visits):
    # Full performance kinematic model
    rotator = rotator_movement(100)
    kinematic_model_ideal = KinemModel(mjd0=visits.iloc[0].obs_start_mjd - 0.1, telescope="comcam")
    kinematic_model_ideal.setup_camera(filter_changetime=90)
    kinematic_model_ideal.mounted_filters = ['u', 'g', 'r', 'i', 'z', 'y']
    kinematic_model_ideal.cumulative_azimuth_rad = 0
    # TMA Event Maker
    eventMaker = TMAEventMaker()
    efd_client = EfdClient('usdf_efd')
    
    model_slewtimes = {}     # current/degraded performance model
    model_slewtimes_ideal = {} # normal performance model
    tma_slews = {}
    tma_tracks = {}
    tma_slewids = {}
    n_tmaevents = {}
    n_tmafaults = {}
    n_tmastops = {}
    image_tracking = {}
    wait_time_after = {}
    wait_time_before = {}
    
    for dayobs in day_obs_list:
        day = int(dayobs.replace('-', ''))
        night_visits = visits.query('day_obs == @day').sort_values(by='seq_num')
        if len(night_visits) > 0:
            t_start = Time(night_visits.obs_start_mjd.min(), format='mjd', scale='tai').utc
            t_end = Time(night_visits.obs_end_mjd.max(), format='mjd', scale='tai').utc
            tma = await get_tma_limits(t_start, t_end, efd_client)
            tma = dict(tma.iloc[0]) # .median())
            tma['settle_time'] = 0
            print(dayobs, tma)
            # Set up current kinematic model.         
            kinematic_model = KinemModel(mjd0=night_visits.iloc[0].obs_start_mjd - 0.1, telescope="comcam")
            kinematic_model.setup_telescope(**tma)
            kinematic_model.setup_camera(filter_changetime=90, **rotator)
            kinematic_model.mounted_filters = ['u', 'g', 'r', 'i', 'z', 'y']
            kinematic_model.cumulative_azimuth_rad = 0
            kinematic_model.park()
            for visitid, v in night_visits.iterrows():
                if np.isnan(v.s_ra) | np.isnan(v.s_dec):
                    model_slewtimes[visitid] = np.nan
                else:
                    ra_rad = [np.radians(v.s_ra)]
                    dec_rad = [np.radians(v.s_dec)]
                    sky_angle = [np.radians(v.sky_rotation)]
                    mjd = v.obs_start_mjd
                    band = v.band
                    slewtime = kinematic_model.slew_times(ra_rad, dec_rad, mjd, rot_sky_pos=sky_angle, filtername=band, update_tracking=True)
                    try:
                        model_slewtimes[visitid] = slewtime[0]
                    # Sometimes it doesn't return an array, just a float ..
                    except TypeError:
                        model_slewtimes[visitid] = slewtime
    
            kinematic_model_ideal.park()
            for visitid, v in night_visits.iterrows():
                if np.isnan(v.s_ra) | np.isnan(v.s_dec):
                    model_slewtimes_ideal[visitid] = np.nan
                else:
                    ra_rad = [np.radians(v.s_ra)]
                    dec_rad = [np.radians(v.s_dec)]
                    sky_angle = [np.radians(v.sky_rotation)]
                    mjd = v.obs_start_mjd
                    band = v.band
                    slewtime = kinematic_model_ideal.slew_times(ra_rad, dec_rad, mjd, rot_sky_pos=sky_angle, filtername=band, update_tracking=True)
                    try:
                        model_slewtimes_ideal[visitid] = slewtime[0]
                    # Sometimes it doesn't return an array, just a float ..
                    except TypeError:
                        model_slewtimes_ideal[visitid] = slewtime
            
            events = eventMaker.getEvents(day, addBlockInfo=False)                
            for i, s in night_visits.iterrows():
                slewtime = 0
                tracktime = 0
                wa = 100000
                wb = 100000
                nevents = 0
                n_stops = 0
                n_faults = 0
                slewids = []
                image_tracking[i] = False
                for e in events:
                    if e.begin.tai.mjd > s.prev_obs_end_mjd and e.end.tai.mjd < s.obs_start_mjd:
                        if e.type == TMAState.SLEWING:                        
                            # There could be more than one slew event between visits
                            nevents += 1
                            slewids.append(e.seqNum)
                            slewtime += e.duration
                            dt = (e.begin.tai.mjd - s.prev_obs_end_mjd) * 24 * 60 * 60
                            wb = np.min([dt, wb])
                            dt = (s.obs_start_mjd - e.end.tai.mjd) * 24 * 60 * 60
                            wa = np.min([dt, wa])
                        if e.endReason == TMAState.STOPPED:
                            n_stops += 1
                        if e.endReason == TMAState.FAULT:
                            n_faults += 1
                    if e.begin.tai.mjd <= s.obs_start_mjd and e.end.tai.mjd >= s.obs_end_mjd:
                        if e.type == TMAState.TRACKING:
                            tracktime += e.duration
                            image_tracking[i]= True
                tma_slews[i] = slewtime
                tma_slewids[i] = slewids
                n_tmaevents[i] = nevents
                n_tmafaults[i] = n_faults
                n_tmastops[i] = n_stops
                tma_tracks[i] = tracktime
                wait_time_after[i] = wa
                wait_time_before[i] = wb

    slewing = pd.DataFrame([tma_slewids, tma_slews, tma_tracks, model_slewtimes, model_slewtimes_ideal, wait_time_before, wait_time_after, n_tmaevents, n_tmafaults, n_tmastops, image_tracking], 
                           index=['TMAseqnums', 'TMAslew', 'TMAtrack', 'slewModel', 'slewModelIdeal', 'waitBeforeSlew', 'waitAfterSlew', 'nTMAslews', 'nTMAfaults', 'nTMAstops', 'image_tracking']).T
    
    return slewing

In [ ]:
# Split up requests by periods of particular TMA speed 
day_obs_min = "2024-10-24"
day_obs_int_min = day_obs_min.replace('-', '')
day_obs_max = "2024-12-12"
day_obs_int_max = day_obs_max.replace('-', '')
print(day_obs_min, day_obs_max)
day_min = Time(f"{day_obs_min}T12:00:00", format='isot', scale='utc')
day_max = Time(f"{day_obs_max}T12:00:00", format='isot', scale='utc')
one_day = TimeDelta(1, format='jd')
days = day_min + one_day * np.arange(0, (day_max - day_min).jd + 1)
day_obs_list = [d.iso[0:10] for d in days]

In [ ]:
# Get Visit Information. 
visits = get_visits(day_obs_int_min, day_obs_int_max)

In [ ]:
tracking = await get_slewtimes(day_obs_list, visits)
visitT = visits.merge(tracking, right_index=True, left_index=True)
print(len(visitT), len(tracking))

In [ ]:
visitS = visitT.query('TMAslew > 0').copy()
visitS['alt'] = 90 - visitS['altitude']
visitS['exp_midpt'] = pd.to_datetime(visitS['exp_midpt'], format='mixed')
scols = ['waitBeforeSlew', 'waitAfterSlew', 'nTMAslews', 'nTMAfaults', 'nTMAstops', 'TMAslew', 'slewModel', 'slewModelIdeal', 'delta_t', 'band', 'img_type', 'science_program', 'target_name', 'observation_reason', 
         's_ra', 's_dec', 'sky_rotation', 'alt', 'azimuth']

In [ ]:
#visitS = visitS.query('waitBeforeSlew < 1000')
#visitS.query('delta_t > 60 * 10')[scols]
#visitS.to_hdf('slew_dataframe.h5', key='visits')

In [ ]:
plt.figure(figsize=(8, 6))
# Pull up slews within science_program BLOCK-320
within_program = np.where(visitS.iloc[:-1].science_program.values == visitS.iloc[1:].science_program.values)[0]
q = visitS.iloc[within_program + 1]
q = q.query('science_program == "BLOCK-T345"')

plt.scatter(q.slewModel, q.TMAslew, 
            c=(q.exp_midpt_mjd - q.exp_midpt_mjd.min()), 
            s=7)
plt.colorbar(label='days after start of commissioning')
x = np.arange(0, 100)
plt.plot(x, x, 'b:')
plt.plot(x, x+0.9, 'r:')
plt.xlim(0, 7)
plt.ylim(0, 7)
plt.xlabel("Slew Model (seconds)", fontsize='x-large')
plt.ylabel("TMA Event SLEW (seconds)", fontsize='x-large')
plt.grid(alpha=0.3)
plt.title("TMAevent SLEW vs. Model Slew BLOCK-T345", fontsize='large')
plt.savefig('SlewModelVsTMA.png')

In [ ]:
q[scols]

In [ ]:
plt.figure(figsize=(8, 6))
# Pull up slews within science_program BLOCK-320
within_program = np.where(visitS.iloc[:-1].science_program.values == visitS.iloc[1:].science_program.values)[0]
q = visitS.iloc[within_program + 1]
q = q.query('science_program == "BLOCK-320"')# or science_program == "PP-SURVEY"')
#q = visitS.query('science_program == "BLOCK-320"')# or science_program == "BLOCK-T345"')
plt.scatter(q.slewModel, q.TMAslew, 
            c=(q.exp_midpt_mjd - q.exp_midpt_mjd.min()), 
            s=7)
plt.colorbar(label='days after start of commissioning')
x = np.arange(0, 100)
plt.plot(x, x, 'b:')
plt.plot(x, x+0.9, 'r:')
plt.xlim(0, 7)
plt.ylim(0, 7)
plt.xlabel("Slew Model (seconds)", fontsize='x-large')
plt.ylabel("TMA Event SLEW (seconds)", fontsize='x-large')
plt.grid(alpha=0.3)
plt.title("TMAevent SLEW vs. Model Slew BLOCK-320", fontsize='large')
plt.savefig('SlewModelVsTMA.png')

In [ ]:
dd = q.groupby('day_obs').agg({'delta_t': ['median', 'mean', 'min', 'max'], 'exp_midpt': 'first', 'TMAslew': ['median', 'mean'], 'waitBeforeSlew': ['median', 'mean'], 'waitAfterSlew': ['median', 'mean']})

x = np.arange(0, len(dd))
plt.figure(figsize=(10, 6))
plt.bar(x, dd['delta_t', 'median'], edgecolor='k', facecolor='white', linewidth=2, label='Median time between visits')

y = dd['waitBeforeSlew', 'median']
plt.bar(x, y, alpha=0.6, edgecolor=None, label="Median wait before TMA slew")
ys = dd['TMAslew', 'median']
plt.bar(x, ys, bottom=y, alpha=0.6, edgecolor=None, label="Median TMA slew")
ya = dd['waitAfterSlew', 'median']
plt.bar(x, ya, bottom=y + ys, alpha=0.6, edgecolor=None, label="Median wait after TMA slew")
_ = plt.xticks(ticks=x, labels=dd.index, rotation=45)
plt.xlim(-1, len(dd))
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(2)) 
plt.legend()
plt.xlabel("DayObs", fontsize='x-large')
plt.ylabel("Time (seconds)", fontsize='x-large')
plt.title("Time between visits during BLOCK-320", fontsize='large')
plt.savefig('TimeBetweenVisits.png')

In [ ]:
q.query('day_obs == 20241115')[scols]

In [ ]:
# within_program = np.where(visitS.iloc[:-1].science_program.values == visitS.iloc[1:].science_program.values)[0]
# q = visitS.iloc[within_program + 1]
# q = q.query('science_program == "BLOCK-320"')
# q.query('TMAslew > 5 and slewModel < 3')[scols]

In [ ]:
q[scols]

In [ ]:
q.query('seq_num >= 564 and seq_num <= 565')['obs_end']

In [ ]:
plt.figure()
block = "BLOCK-T345"
q = visitS.query('science_program == @block')
qa = q.query('band == "r"')
plt.scatter(qa.azimuth, qa.alt, c=qa.seq_num, marker='o')
plt.plot(qa.azimuth, qa.alt, linestyle=':')
qb = q.query('band == "g"')
plt.scatter(qb.azimuth, qb.alt, c=qb.seq_num, marker='x')
plt.plot(qb.azimuth, qb.alt, linestyle=':')
plt.colorbar(label='seq_num')
plt.ylabel("Altitude (degrees)")
plt.xlabel("Azimuth (degrees)")
plt.grid(alpha=0.3)

plt.figure()
q = visitS.query('science_program == @block')
qa = q.query('band == "r"')
plt.scatter(qa.s_ra, qa.s_dec, c=qa.seq_num, marker='o')
plt.plot(qa.s_ra, qa.s_dec, linestyle=':')
qb = q.query('band == "g"')
plt.scatter(qb.s_ra, qb.s_dec, c=qb.seq_num, marker='x')
plt.plot(qb.s_ra, qb.s_dec, linestyle=':')
plt.colorbar(label='seq_num')
plt.ylabel("RA (degrees)")
plt.xlabel("Dec (degrees)")
plt.grid(alpha=0.3)

In [ ]:
distances = angular_separation(q.s_ra.values[1:], q.s_dec.values[1:], q.s_ra.values[:-1], q.s_dec.values[:-1])
plt.plot(distances, q.waitAfterSlew.values[1:], 'k.', label='wait after slew')
plt.plot(distances, q.waitBeforeSlew.values[1:], 'r.', label='wait before slew')
plt.legend()
plt.xlabel("Slew distance (degrees)", fontsize='large')
plt.ylabel("slew wait time (seconds)", fontsize='large')
plt.title("BLOCK-T345", fontsize='large')
plt.ylim(0, 160)
plt.grid(alpha=0.3)

In [ ]:
plt.figure(figsize=(8,6))
block = "BLOCK-320"
#block = "BLOCK-T345"
q = visitS.query('science_program == @block')
for day_obs in day_obs_list:
    dayobs = int(day_obs.replace('-', ''))
    m = visitS.query('day_obs == @dayobs and science_program == @block')
    if len(m) > 0:
        _ = plt.hist(m.waitAfterSlew, bins=np.arange(0, 30, 0.5), histtype='step', label=f'dayobs {dayobs}')
        print(dayobs, m.waitAfterSlew.mean(), m.waitAfterSlew.median(), m.target_name.unique())
plt.legend(loc=(1.01, 0.1))
plt.xlabel("Time (seconds)", fontsize='large')
plt.grid(alpha=0.3)
_ = plt.title('Time after TMA SLEWING', fontsize='large')

In [ ]:
plt.figure(figsize=(8,6))
q = visitS.query('science_program == @block')
for day_obs in day_obs_list:
    dayobs = int(day_obs.replace('-', ''))
    m = visitS.query('day_obs == @dayobs and science_program == @block')
    if len(m) > 0:
        _ = plt.hist(m.waitBeforeSlew, bins=np.arange(0, 80, 0.5), histtype='step', label=f'dayobs {dayobs}')
        print(dayobs, m.waitBeforeSlew.mean(), m.waitBeforeSlew.median(), m.target_name.unique())
plt.legend(loc=(1.01, 0.1))
plt.xlabel("Time (seconds)", fontsize='large')
plt.grid(alpha=0.3)
_ = plt.title('Time before TMA SLEWING', fontsize='large')

In [ ]:
plt.figure(figsize=(8,6))
q = visitS.query('science_program == @block')
for day_obs in day_obs_list:
    dayobs = int(day_obs.replace('-', ''))
    m = visitS.query('day_obs == @dayobs and science_program == @block')
    if len(m) > 0:
        _ = plt.hist(m.TMAslew, bins=np.arange(0, 20, 0.5), histtype='step', label=f'dayobs {dayobs}')
        print(dayobs, m.TMAslew.mean(), m.TMAslew.median(), m.target_name.unique())
plt.legend(loc=(1.01, 0.1))
plt.xlabel("Time (seconds)", fontsize='large')
plt.grid(alpha=0.3)
_ = plt.title('Time of TMA SLEWING', fontsize='large')

In [ ]:
visitS.query('science_program == @block')[scols]

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(visitS.rotTelPos_vel, visitS.delta_t, 'k.', label='Delta T (between exposures)')
plt.plot(visitS.rotTelPos_vel, visitS.waitAfterSlew, 'r.', label="Wait after TMA slew")
plt.plot(visitS.rotTelPos_vel, visitS.waitBeforeSlew, 'b.', label="Wait before TMA slew")
plt.legend(loc=(1.01, 0.5))
plt.xlabel("necessary rotator velocity", fontsize='large')
_ = plt.ylabel("Time (seconds)", fontsize='large')
plt.ylim(0, 400)

In [ ]:
plt.figure(figsize=(8, 6))
q = visitS.query('day_obs == 20241208')
print(q.target_name.unique())
plt.scatter(q.rotTelPos_vel, q.waitBeforeSlew, c=q.alt, marker='x', label="Wait before TMA slew")
plt.scatter(q.rotTelPos_vel, q.waitAfterSlew, c=q.alt, label="Wait after TMA slew")
plt.colorbar(label='alt')
q = q.query('nTMAfaults > 0 or nTMAstops > 0')
plt.scatter(q.rotTelPos_vel, q.waitBeforeSlew, c='r', marker='+')
plt.scatter(q.rotTelPos_vel, q.waitAfterSlew, c='r', marker='+')
#plt.legend()
plt.xlabel("necessary rotator velocity", fontsize='large')
plt.ylabel("Time (seconds)", fontsize='large')
plt.grid(alpha=0.3)
plt.legend()
#plt.xlim(-0.015, 0.015)
#plt.ylim(-1, 300)

In [ ]:
plt.figure(figsize=(8, 6))
q = visitS.query('science_program == "BLOCK-320"')
print(q.target_name.unique())
plt.scatter(q.rotTelPos_vel, q.waitBeforeSlew, c=q.alt, marker='x', label="Wait before TMA slew")
plt.scatter(q.rotTelPos_vel, q.waitAfterSlew, c=q.alt, label="Wait after TMA slew")
plt.colorbar(label='alt')
q = q.query('nTMAfaults > 0 or nTMAstops > 0')
plt.scatter(q.rotTelPos_vel, q.waitBeforeSlew, c='r', marker='+')
plt.scatter(q.rotTelPos_vel, q.waitAfterSlew, c='r', marker='+')
#plt.legend()
plt.xlabel("necessary rotator velocity", fontsize='large')
plt.ylabel("Time (seconds)", fontsize='large')
plt.grid(alpha=0.3)
plt.legend()
plt.ylim(0, 300)

In [ ]:
fig, axs  = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(8, 20))

# Look at a newer night 
# obs for science? 

dayobs = 20241208
print(visitS.query('science_program == "BLOCK-320" and day_obs == @dayobs').target_name.unique())

seq = visitS.query('science_program == "BLOCK-320" and target_name == "ECDFS" and day_obs == @dayobs')
#seq = visitS.query('science_program == "BLOCK-320" and target_name == "Rubin_SV_095_-25" and day_obs == @dayobs')
#dayobs = 20241125	
#seq = visitT.query('science_program == "BLOCK-320" and target_name == "EDFS_comcam" and day_obs == @dayobs')
seq_start = seq.iloc[0].seq_num # 150
seq_end = seq.iloc[-1].seq_num # 180

vv = visitT.query('day_obs == @dayobs and seq_num > @seq_start and seq_num < @seq_end')
times = vv.obs_start_mjd.values 
times = (times - times[0]) * 24 * 60

ax = axs[0]
ax.plot(times, vv.delta_t, 'teal', marker='.', label='Delta T Visits')
ax.plot(times, vv.slewModel, 'orange', marker='.', label="Predicted Delta T Visit")
ax.plot(times, vv.TMAslew, 'red', marker='.', label="TMA Event Slews")
filterchanges = np.where(vv['band'][1:].values != vv['band'][:-1].values)[0] + 1
for filterchange in filterchanges[0:1]:
    ax.axvline(times[filterchange], color='k', linestyle=':', label='filter change')
for filterchange in filterchanges[1:]:
    ax.axvline(times[filterchange], color='k', linestyle=':')
for t in times[0:1]:
    ax.axvline(t, color='r', linestyle=':', alpha=0.3, label='exposure start')
for t in times[1:]:
    ax.axvline(t, color='r', linestyle=':', alpha=0.3)
ax.set_xlabel("Time from start of visits (minutes)", fontsize='x-large')
ax.set_ylabel("Time (seconds)", fontsize='x-large')
ax.legend(loc=(1.01, 0.2))
_ = ax.set_title(f"Slewtimes on {dayobs} between seq_nums {seq_start} - {seq_end}")

ax = axs[1]
ax.plot(times, vv.slewModel - vv.TMAslew, 'black', marker='.', label='Model - TMAEvent Difference')
ax.axhline(3, color='teal', linestyle='-.', label="TMA Settle (+3s in model slewtime)")
for filterchange in filterchanges[0:1]:
    ax.axvline(times[filterchange], color='k', linestyle=':', label='filter change')
for filterchange in filterchanges[1:]:
    ax.axvline(times[filterchange], color='k', linestyle=':')
for t in times[0:1]:
    ax.axvline(t, color='r', linestyle=':', alpha=0.3, label='exposure')
for t in times[1:]:
    ax.axvline(t, color='r', linestyle=':', alpha=0.3)
ax.set_xlabel("Time from start of sequence (minutes)", fontsize='x-large')
ax.set_ylabel("Difference", fontsize='x-large')
ax.set_ylim(0, 5)
ax.legend(loc=(1.01, 0.2))
#_ = ax.set_title(f"Slewtimes on {dayobs} between seq_nums {seq_start} - {seq_end}")

ax = axs[2]
ax.plot(times, vv.delta_t - vv.TMAslew, 'black', marker='.', label='Dt - TMAEvent Difference')
ax.plot(times, vv.delta_t - vv.slewModel, 'red', marker='.', linestyle='-.', label='Dt - Model Difference')
ax.axhline(13.5, color='teal', linestyle='-.', label="Extra TMA Settle (13.5s)")
for filterchange in filterchanges[0:1]:
    ax.axvline(times[filterchange], color='k', linestyle=':', label='filter change')
for filterchange in filterchanges[1:]:
    ax.axvline(times[filterchange], color='k', linestyle=':')
for t in times[0:1]:
    ax.axvline(t, color='r', linestyle=':', alpha=0.3, label='exposure')
for t in times[1:]:
    ax.axvline(t, color='r', linestyle=':', alpha=0.3)
ax.set_xlabel("Time from start of sequence (minutes)", fontsize='x-large')
ax.set_ylabel("Difference", fontsize='x-large')
ax.set_ylim(0, 40)
ax.legend(loc=(1.01, 0.2))
#_ = ax.set_title(f"Slewtimes on {dayobs} between seq_nums {seq_start} - {seq_end}")

fig.subplots_adjust(hspace=0)



In [ ]:
# Look at a newer night 
fig, axs  = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(8, 20))

# obs for science? 
dayobs = 20241127
seq = visitT2.query('science_program == "BLOCK-320" and target_name == "ECDFS" and day_obs == @dayobs')
seq_start = seq.iloc[0].seq_num # 150
seq_end = seq.iloc[-1].seq_num # 180
vv = visitT2.query('day_obs == @dayobs and seq_num > @seq_start and seq_num < @seq_end')
times = vv.obs_start_mjd.values 
times = (times - times[0]) * 24 * 60

ax = axs[0]
ax.plot(times, vv.delta_t, 'teal', marker='.', label='Delta T Visits')
ax.plot(times, vv.slewModel, 'orange', marker='.', label="Predicted Delta T Visit")
ax.plot(times, vv.TMAslew, 'red', marker='.', label="TMA Event Slews")
filterchanges = np.where(vv['band'][1:].values != vv['band'][:-1].values)[0] + 1
for filterchange in filterchanges[0:1]:
    ax.axvline(times[filterchange], color='k', linestyle=':', label='filter change')
for filterchange in filterchanges[1:]:
    ax.axvline(times[filterchange], color='k', linestyle=':')
for t in times[0:1]:
    ax.axvline(t, color='r', linestyle=':', alpha=0.3, label='exposure start')
for t in times[1:]:
    ax.axvline(t, color='r', linestyle=':', alpha=0.3)
ax.set_xlabel("Time from start of visits (minutes)", fontsize='x-large')
ax.set_ylabel("Time (seconds)", fontsize='x-large')
ax.legend(loc=(1.01, 0.2))
_ = ax.set_title(f"Slewtimes on {dayobs} between seq_nums {seq_start} - {seq_end}")

ax = axs[1]
ax.plot(times, vv.slewModel - vv.TMAslew, 'black', marker='.', label='Model - TMAEvent Difference')
ax.axhline(3, color='teal', linestyle='-.', label="TMA Settle (+3s in model slewtime)")
for filterchange in filterchanges[0:1]:
    ax.axvline(times[filterchange], color='k', linestyle=':', label='filter change')
for filterchange in filterchanges[1:]:
    ax.axvline(times[filterchange], color='k', linestyle=':')
for t in times[0:1]:
    ax.axvline(t, color='r', linestyle=':', alpha=0.3, label='exposure')
for t in times[1:]:
    ax.axvline(t, color='r', linestyle=':', alpha=0.3)
ax.set_xlabel("Time from start of sequence (minutes)", fontsize='x-large')
ax.set_ylabel("Difference", fontsize='x-large')
ax.set_ylim(0, 5)
ax.legend(loc=(1.01, 0.2))
#_ = ax.set_title(f"Slewtimes on {dayobs} between seq_nums {seq_start} - {seq_end}")

ax = axs[2]
ax.plot(times, vv.delta_t - vv.TMAslew, 'black', marker='.', label='Dt - TMAEvent Difference')
ax.plot(times, vv.delta_t - vv.slewModel, 'red', marker='.', linestyle='-.', label='Dt - Model Difference')
ax.axhline(13.5, color='teal', linestyle='-.', label="Extra TMA Settle (13.5s)")
for filterchange in filterchanges[0:1]:
    ax.axvline(times[filterchange], color='k', linestyle=':', label='filter change')
for filterchange in filterchanges[1:]:
    ax.axvline(times[filterchange], color='k', linestyle=':')
for t in times[0:1]:
    ax.axvline(t, color='r', linestyle=':', alpha=0.3, label='exposure')
for t in times[1:]:
    ax.axvline(t, color='r', linestyle=':', alpha=0.3)
ax.set_xlabel("Time from start of sequence (minutes)", fontsize='x-large')
ax.set_ylabel("Difference", fontsize='x-large')
ax.set_ylim(0, 40)
ax.legend(loc=(1.01, 0.2))
#_ = ax.set_title(f"Slewtimes on {dayobs} between seq_nums {seq_start} - {seq_end}")

fig.subplots_adjust(hspace=0)


In [ ]:
scols = ['waitBeforeSlew', 'waitAfterSlew', 'nTMAslews', 'TMAslew', 'slewModel', 'delta_t', 'band', 'img_type', 'science_program', 'target_name', 'observation_reason', 's_ra', 's_dec', 'sky_rotation']
visitA = pd.concat([visitT, visitT2])

In [ ]:
# Look at science observation periods
sci = visitA.query('science_program == "BLOCK-320"')
# Break down sci into separate sequences, to count values separately per sequence
dd = sci.groupby(['target_name', 'day_obs']).agg({'exposure_name': 'count', 'seq_num': ['min', 'max']})
dd.columns = dd.columns.map('_'.join)
dd = dd.reset_index('day_obs').reset_index('target_name')
dd = dd.rename({'exposure_name_count': 'nexp'}, axis=1)
#dd

In [ ]:
newcols = ['err_count', 'n_filter_changes', 'open_shutter_total', 'seq_time', 'model_slew_total', 'model_ideal_slew_total', 'fault_time', 'wait_before_median', 'wait_after_median', 'tma_slew_median', 'model_slew_median']
zeros = np.zeros(len(dd), float)
tt = pd.DataFrame([zeros] * len(newcols), index=newcols).T
dd = dd.merge(tt, right_index=True, left_index=True)

In [ ]:
efd_client = EfdClient('usdf_efd')
err_list = {}
err_count = []
for i, seq in dd.iterrows():
    vv = visitA.query("seq_num >= @seq.seq_num_min and seq_num <= @seq.seq_num_max and day_obs == @seq.day_obs and target_name == @seq.target_name")
    errs = await get_error_codes(Time(vv.iloc[0].obs_start_mjd, format='mjd', scale='tai').utc, 
                           Time(vv.iloc[-1].obs_end_mjd, format='mjd', scale='tai').utc,
                           efd_client)
    err_list[i] = errs
    err_count.append(len(errs))

def slewtime_sequences(seq, visitA):
    vv = visitA.query("seq_num >= @seq.seq_num_min and seq_num <= @seq.seq_num_max and day_obs == @seq.day_obs and target_name == @seq.target_name")
    # Count up open shutter time
    seq.open_shutter_total = vv.shut_time.sum()
    seq.seq_time = (vv.iloc[-1].obs_end_mjd - vv.iloc[0].obs_start_mjd) * 24 * 60 * 60
    seq.n_filter_changes = len(np.where(vv.band.values[1:] != vv.band.values[:-1])[0])
    seq.tma_slew_median = np.median(vv.TMAslew)
    seq.wait_before_median = np.median(vv.waitBeforeSlew)
    seq.wait_after_median = np.median(vv.waitAfterSlew)
    seq.model_slew_total = vv.slewModel.sum()
    seq.model_ideal_slew_total = vv.slewModelIdeal.sum()
    seq.model_slew_median = np.median(vv.slewModel)
    seq.fault_time = 0
    return seq
tt = dd.apply(slewtime_sequences, args=[visitA], axis=1)

In [ ]:
tt['err_count'] = err_count
tt['tma_event_delays'] = (tt.wait_after_median + tt.wait_before_median)
tt['extra_slew_delay'] = (tt.wait_after_median + tt.wait_before_median + tt.tma_slew_median - tt.model_slew_median)
tt['mod_model_time'] = tt.model_slew_total + tt.nexp * tt.extra_slew_delay   ## problem with filter change time inaccuracies and delay inaccuracy, so futz
tt['model_eff'] = tt.open_shutter_total / (tt.open_shutter_total + tt.mod_model_time) 
tt['model_eff_ideal'] = tt.open_shutter_total / (tt.open_shutter_total + tt.model_ideal_slew_total)
tt['eff'] = tt.open_shutter_total / tt.seq_time
tt.sort_values(by='day_obs')

In [ ]:
tt.sort_values(by='day_obs').tail(8)[['target_name', 'day_obs', 'nexp', 'n_filter_changes', 'wait_before_median', 'wait_after_median', 'tma_slew_median',]]



In [ ]:
# From the errors which don't seem to impact the median slewtime, they're mostly rotator issues or hexapod issues (the 35 errors are also being driven by the electrometer reporting issues)
seq = visitA.query('target_name == "ECDFS" and day_obs == 20241127')[['band', 'nTMAslews', 'waitBeforeSlew', 'waitAfterSlew', 'TMAslew', 'slewModel', 'slewModelIdeal', 'delta_t', 's_ra', 's_dec', 'sky_rotation']]
seq = seq.query('slewModel < 90')
plt.scatter(seq.TMAslew, seq.waitAfterSlew) #, c=seq.sky_rotation)
plt.xlabel("TMA slew time")
plt.ylabel("Wait After Slew")

In [ ]:
plt.figure(figsize=(10, 6))
x = tt.sort_values(by='day_obs')[4:].query('nexp > 5')
xval = (x.day_obs - 20241100).values
yval = (x.eff/ x.model_eff_ideal).values
plt.scatter(xval, yval, c=x.n_filter_changes, cmap='tab10', s=x.nexp*1.5)
plt.colorbar(orientation='vertical', label="N filter changes")
for i, (text, xi, yi) in enumerate(zip(x.target_name.values, xval, yval)):
    if (i-2) % 3 == 0:        
        yoffset = 0.015 
    else:
        yoffset = -0.015
    if (i-1) % 3 == 0:
        xoffset = 0.15
    else:
        xoffset = 0.1
    plt.annotate(text, (xi + xoffset, yi + yoffset))
# for i in np.arange(15.5, 28.5, 1):
#     plt.axvspan(i, i+1, color='gray', alpha=0.7-(i-15)/20)
# plt.axhline(1, color='k', linestyle=':')
plt.grid(alpha=0.4)
plt.ylim(0.3, 1.0)
plt.xticks(np.arange(16, 29))
#plt.xlim(16.5, 27.5)
plt.xlabel("Day of November (dayobs 202411-x)", fontsize='large')
_ = plt.ylabel("OpenShutterFraction / Modeled OpenShutterFraction", fontsize='large')


In [ ]:
x.tail()[['target_name', 'day_obs', 'nexp', 'n_filter_changes', 'open_shutter_total', 'seq_time', 'model_ideal_slew_total', 'model_slew_total',  'model_eff_ideal', 'model_eff', 'eff']]

In [ ]:
plt.plot(x.day_obs, x.model_eff_ideal, 'k.')
plt.plot(x.day_obs, x.model_eff, 'r.')
plt.plot(x.day_obs, x.eff, 'b+')

In [ ]:
efd_client = EfdClient('usdf_efd')

In [ ]:
tma_movement(10)

In [ ]:
rotator_movement(100)